In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [4]:
device = 'mps' if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)  # Вход: 32x32, Выход: 32x32
        self.norm1 = nn.BatchNorm2d(32)
        self.drop1 = nn.Dropout2d(p=0.2)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)  # Вход: 32x32, Выход: 32x32
        self.norm2 = nn.BatchNorm2d(64)
        
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Вход: 32x32, Выход: 16x16
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)  # Вход: 16x16, Выход: 16x16
        self.norm3 = nn.BatchNorm2d(128)
        
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Вход: 16x16, Выход: 8x8
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1)  # Вход: 8x8, Выход: 6x6
        self.norm4 = nn.BatchNorm2d(64)
        
        self.pool3 = nn.MaxPool2d(kernel_size=4)  # Вход: 6x6, Выход: 1x1
        self.drop2 = nn.Dropout2d(p=0.2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=64, out_features=256)
        self.drop3 = nn.Dropout(p=0.1)
        self.fc2 = nn.Linear(in_features=256, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.norm1(x)
        x = self.drop1(x)
        
        x = F.relu(self.conv2(x))
        x = self.norm2(x)
        
        x = self.pool1(x)
        x = F.relu(self.conv3(x))
        x = self.norm3(x)
        
        x = self.pool2(x)
        x = F.relu(self.conv4(x))
        x = self.norm4(x)
        
        x = self.pool3(x)
        x = self.drop2(x)
        
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.drop3(x)
        x = self.fc2(x)
        return x

#### Code

In [2]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
batch_size=32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
train_loader = DataLoader(train_dataset, shuffle=False, num_workers=2)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=400, out_features=120)
        self.fc2 = nn.Linear(in_features=400, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x      

model = Net()
model.to(device)
loss_func = nn.CrossEntropyLoss()
opt = optim.Adam(params=model.parameters(), lr=0.001)
scheduler = lr_scheduler.MultiStepLR(opt, milestones=[10, 13], gamma=0.5)

In [6]:
def fit(model, opt, loss_func, train_loader, epochs, scheduler):
    history_loss = []

    for epoch in tqdm(range(epochs)):
        model.train()
        loss_sum = 0
        for xb, yb in tqdm(train_loader):
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            loss_sum += loss.item()
        history_loss.append(loss_sum / len(train_loader))
        scheduler.step()
        print(f'Epoch {epoch+1}: Train Loss = {history_loss[-1]},  Lr = {scheduler.get_last_lr()}')
        
    return history_loss

In [ ]:
epochs = 16
hist = fit(model, opt, loss_func, train_loader, epochs, scheduler)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

#### predict 

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_folder = ImageFolder(root='./data/test_up/', transform=transform)
test_loader = DataLoader(test_folder, shuffle=False)

In [17]:
# model = torch.load('./models/model_CIFAR_10_1.pth', map_location=torch.device('cpu'))
# import torch
# import YourModelClass
# model = YourModelClass() 
# checkpoint = torch.load('./models/model_CIFAR_10_1.pth', map_location=torch.device('cpu'))
# model.load_state_dict(checkpoint)

file_path = './models/model_CIFAR_10_1.pth'
model = Net()
checkpoint = torch.load(file_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [18]:
y_pred = []
model.eval()
with torch.no_grad():
    for x, _ in tqdm(test_loader):
        output = model(x)
        logit = F.softmax(output, dim=1)
        _, pred = torch.max(logit, dim=1)
        y_pred.append(pred.item())        

  0%|          | 0/300000 [00:00<?, ?it/s]

In [22]:
df_sub = pd.read_csv('./data/sampleSubmission.csv')
df_sub['label'] = y_pred
df_sub.to_csv('CIFAR_ans.csv', index=False)

In [2]:
df_ans = pd.read_csv('CIFAR_ans.csv')

In [4]:
class_name = ["airplane", "automobile", "bird","cat", "deer", "dog","frog","horse", "ship","truck"]

In [11]:
label_list = df_ans['label'].tolist()

# Use the label_list as an index for class_name
indexed_class_name = [class_name[label] for label in label_list]

In [13]:
df_ans['label'] = indexed_class_name

In [15]:
df_ans
df_ans.to_csv('CIFAR_ans_name.csv', index=False)